In [1]:
import csv

import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

RANDOM_SEED = 42

# Specify each path

In [16]:
dataset = 'model/keypoint_classifier_v1/keypoint_classifier_v1.csv'
model_save_path = 'model/keypoint_classifier_v1/keypoint_classifier_v1.keras'


# Classification number setting

In [3]:
NUM_CLASSES = 19

# Loading training data

In [21]:
from sklearn.preprocessing import LabelEncoder

# Đọc dữ liệu dạng chuỗi
y_dataset = np.loadtxt(dataset, delimiter=',', dtype='str', usecols=(0))

# Mã hóa nhãn
label_encoder = LabelEncoder()
y_dataset_encoded = label_encoder.fit_transform(y_dataset)

print(y_dataset_encoded)


[ 0  0  0 ... 18 18 18]


In [7]:
X_dataset = np.loadtxt(dataset, delimiter=',', dtype='float32', usecols=list(range(1, (21 * 2) + 1)))

In [22]:
y_dataset = y_dataset_encoded

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X_dataset, y_dataset, train_size=0.75, random_state=RANDOM_SEED)

# Model building

In [24]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Input((21 * 2, )),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(20, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')
])

In [25]:
model.summary()  # tf.keras.utils.plot_model(model, show_shapes=True)

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dropout_2 (Dropout)             │ (None, 42)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 20)             │           860 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 20)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 10)             │           210 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 19)             │           209 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,279 (5.00 KB)

 Trainable params: 1,279 (5.00 KB)

 Non-trainable params: 0 (0.00 B)

In [26]:
# Model checkpoint callback
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    model_save_path, verbose=1, save_weights_only=False)
# Callback for early stopping
es_callback = tf.keras.callbacks.EarlyStopping(patience=20, verbose=1)

In [27]:
# Model Compilation
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Model Training

In [28]:
model.fit(
    X_train,
    y_train,
    epochs=1000,
    batch_size=128,
    validation_data=(X_test, y_test),
    callbacks=[cp_callback, es_callback]
)

Epoch 1/1000


 1/12 ━━━━━━━━━━━━━━━━━━━━ 18s 2s/step - accuracy: 0.0547 - loss: 2.9344
Epoch 1: saving model to model/keypoint_classifier_v1/keypoint_classifier_v1.keras
12/12 ━━━━━━━━━━━━━━━━━━━━ 2s 33ms/step - accuracy: 0.0692 - loss: 2.9206 - val_accuracy: 0.1010 - val_loss: 2.8535
Epoch 2/1000
 1/12 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - accuracy: 0.0391 - loss: 2.8806
Epoch 2: saving model to model/keypoint_classifier_v1/keypoint_classifier_v1.keras
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.0712 - loss: 2.8722 - val_accuracy: 0.0990 - val_loss: 2.8158
Epoch 3/1000
 1/12 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - accuracy: 0.0859 - loss: 2.8043
Epoch 3: saving model to model/keypoint_classifier_v1/keypoint_classifier_v1.keras
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.0882 - loss: 2.8099 - val_accuracy: 0.1455 - val_loss: 2.7737
Epoch 4/1000
 1/12 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - accuracy: 0.1094 - loss: 2.7788
Epoch 4: saving model to model/keypoint_classifier_v1/keypoint_classi

In [29]:
# Model evaluation
val_loss, val_acc = model.evaluate(X_test, y_test, batch_size=128)

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8233 - loss: 0.7512


In [30]:
# Loading a saved model
model = tf.keras.models.load_model(model_save_path)

In [31]:
# Inference Test
predict_result = model.predict(np.array([X_test[0]]))
print(np.squeeze(predict_result))
print(np.argmax(np.squeeze(predict_result)))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
[3.50500952e-04 6.55816335e-09 9.22443578e-05 2.30096632e-07
 1.07065663e-02 3.24059397e-01 8.36030811e-12 2.65990701e-02
 6.07994080e-12 2.08277052e-04 5.12509679e-09 5.63810110e-09
 8.20920570e-04 8.44341144e-03 1.00508405e-04 1.23659542e-04
 3.35337967e-01 2.92975217e-01 1.82076808e-04]
16


# Mix rows and columns

In [32]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report

def print_confusion_matrix(y_true, y_pred, report=True):
    labels = sorted(list(set(y_true)))
    cmx_data = confusion_matrix(y_true, y_pred, labels=labels)
    
    df_cmx = pd.DataFrame(cmx_data, index=labels, columns=labels)
 
    fig, ax = plt.subplots(figsize=(7, 6))
    sns.heatmap(df_cmx, annot=True, fmt='g' ,square=False)
    ax.set_ylim(len(set(y_true)), 0)
    plt.show()
    
    if report:
        print('Classification Report')
        print(classification_report(y_test, y_pred))

Y_pred = model.predict(X_test)
y_pred = np.argmax(Y_pred, axis=1)

print_confusion_matrix(y_test, y_pred)

ModuleNotFoundError: No module named 'seaborn'

# Convert to a model for Tensorflow-Lite

In [33]:
# Save as inference-only model
model.save(model_save_path, include_optimizer=False)

In [34]:
# Transform the model (quantize it)
tflite_save_path = 'model/keypoint_classifier_v1/keypoint_classifier_v1.tflite'

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quantized_model = converter.convert()

open(tflite_save_path, 'wb').write(tflite_quantized_model)

INFO:tensorflow:Assets written to: C:\Users\admin\AppData\Local\Temp\tmpybhttio8\assets


INFO:tensorflow:Assets written to: C:\Users\admin\AppData\Local\Temp\tmpybhttio8\assets


Saved artifact at 'C:\Users\admin\AppData\Local\Temp\tmpybhttio8'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 42), dtype=tf.float32, name='input_layer_1')
Output Type:
  TensorSpec(shape=(None, 19), dtype=tf.float32, name=None)
Captures:
  1686884284944: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1686884843792: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1686884843984: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1686884858576: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1686884844176: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1686884848592: TensorSpec(shape=(), dtype=tf.resource, name=None)


7200

# Inference test

In [35]:
interpreter = tf.lite.Interpreter(model_path=tflite_save_path)
interpreter.allocate_tensors()

In [36]:
# Get input and output tensors
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [37]:
interpreter.set_tensor(input_details[0]['index'], np.array([X_test[0]]))

In [38]:
%%time
# Inference execution
interpreter.invoke()
tflite_results = interpreter.get_tensor(output_details[0]['index'])

CPU times: total: 0 ns
Wall time: 1.01 ms


In [39]:
print(np.squeeze(tflite_results))
print(np.argmax(np.squeeze(tflite_results)))

[3.5050145e-04 6.5581749e-09 9.2244351e-05 2.3009684e-07 1.0706571e-02
 3.2405937e-01 8.3603393e-12 2.6599079e-02 6.0799629e-12 2.0827721e-04
 5.1250963e-09 5.6381109e-09 8.2092010e-04 8.4434105e-03 1.0050840e-04
 1.2365963e-04 3.3533794e-01 2.9297519e-01 1.8207697e-04]
16
